# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.243/10508/1 Dashboard: http://10.20.0.243:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1046,Norbert,-0.9429070228101801,-0.275210207817838
2000-01-01 00:00:01,973,Kevin,0.544917899414675,0.03081765607585929
2000-01-01 00:00:02,1005,Norbert,-0.9834433664495212,0.22777440505401603
2000-01-01 00:00:03,989,Tim,0.8635323314523757,0.20729967412755723
2000-01-01 00:00:04,995,Michael,0.7105854787848316,0.0065984489176820205
2000-01-01 00:00:05,1016,Hannah,-0.9714041924748349,-0.2800877032513167
2000-01-01 00:00:06,1025,Patricia,-0.9491968948866691,-0.734149507755445
2000-01-01 00:00:07,948,Dan,0.4019481125407527,-0.1951675757046898
2000-01-01 00:00:08,982,George,-0.7727975249751413,-0.5491397757782905


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,988,Tim,0.03894941078297354,0.01982638244382695
2000-01-30 00:00:01,958,Kevin,-0.6083676010008399,-0.4020947203408598
2000-01-30 00:00:02,980,Dan,-0.5654592770259843,-0.5082820470435998
2000-01-30 00:00:03,1035,Zelda,-0.2136811261240723,-0.2114070659138203
2000-01-30 00:00:04,931,Bob,-0.9288450073810441,0.6117475493674898
2000-01-30 00:00:05,956,Tim,0.39485926369635926,0.5446446829087386
2000-01-30 00:00:06,980,Ray,0.669021695156107,-0.9390892661932548
2000-01-30 00:00:07,1028,Patricia,-0.5890715738813452,-0.8041584332494649
2000-01-30 00:00:08,1039,Michael,0.7883876579701055,-0.8551952196968697


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1046,Norbert,-0.942907,-0.275210
1,2000-01-01 00:00:01,973,Kevin,0.544918,0.030818
2,2000-01-01 00:00:02,1005,Norbert,-0.983443,0.227774
3,2000-01-01 00:00:03,989,Tim,0.863532,0.207300
4,2000-01-01 00:00:04,995,Michael,0.710585,0.006598


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1046,Norbert,-0.942907,-0.275210
1,2000-01-01 00:00:01,973,Kevin,0.544918,0.030818
2,2000-01-01 00:00:02,1005,Norbert,-0.983443,0.227774
3,2000-01-01 00:00:03,989,Tim,0.863532,0.207300
4,2000-01-01 00:00:04,995,Michael,0.710585,0.006598


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.6 s, sys: 612 ms, total: 5.21 s
Wall time: 3.38 s


name
Alice       0.001087
Bob        -0.003849
Charlie    -0.001899
Dan         0.001535
Edith      -0.000830
Frank       0.002782
George      0.000208
Hannah      0.000890
Ingrid      0.002917
Jerry       0.001132
Kevin      -0.000027
Laura       0.001293
Michael     0.001206
Norbert    -0.002933
Oliver     -0.001659
Patricia    0.000066
Quinn       0.002499
Ray        -0.006340
Sarah      -0.000554
Tim        -0.001172
Ursula     -0.000179
Victor      0.001093
Wendy      -0.001794
Xavier      0.000069
Yvonne     -0.001596
Zelda      -0.001365
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.39 s, sys: 156 ms, total: 1.54 s
Wall time: 1.04 s


name
Alice       0.001087
Bob        -0.003849
Charlie    -0.001899
Dan         0.001535
Edith      -0.000830
Frank       0.002782
George      0.000208
Hannah      0.000890
Ingrid      0.002917
Jerry       0.001132
Kevin      -0.000027
Laura       0.001293
Michael     0.001206
Norbert    -0.002933
Oliver     -0.001659
Patricia    0.000066
Quinn       0.002499
Ray        -0.006340
Sarah      -0.000554
Tim        -0.001172
Ursula     -0.000179
Victor      0.001093
Wendy      -0.001794
Xavier      0.000069
Yvonne     -0.001596
Zelda      -0.001365
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.35 s, sys: 91.2 ms, total: 1.45 s
Wall time: 1.04 s


name
Alice       0.001087
Bob        -0.003849
Charlie    -0.001899
Dan         0.001535
Edith      -0.000830
Frank       0.002782
George      0.000208
Hannah      0.000890
Ingrid      0.002917
Jerry       0.001132
Kevin      -0.000027
Laura       0.001293
Michael     0.001206
Norbert    -0.002933
Oliver     -0.001659
Patricia    0.000066
Quinn       0.002499
Ray        -0.006340
Sarah      -0.000554
Tim        -0.001172
Ursula     -0.000179
Victor      0.001093
Wendy      -0.001794
Xavier      0.000069
Yvonne     -0.001596
Zelda      -0.001365
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html